# Run MPNN interface design on the filtered binders

### Imports

In [5]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
running in directory: /mnt/projects/crispy_shifty
running on node: gpu8


### Set working directory to the root of the crispy_shifty repo

In [6]:
os.chdir("/projects/crispy_shifty")

### Run MPNN on the interfaces

In [4]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "06_mpnn_again"
design_list_file = os.path.join(
    os.getcwd(), "projects/crispy_crispies/05_filter_binders/filtered.list"
)
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")
options = " ".join(
    [
        "out:level 200",
    ]
)
extra_kwargs = {"mpnn_design_area": "full", "num_sequences": "96", "omit_AAs": "CX"}

gen_array_tasks(
    distribute_func="projects.crispy_crispies.deployables.mpnn_binder",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="gpu",
    memory="16G",
    gres="--gres=gpu:a4000:1",
    nstruct=1,
    nstruct_per_task=200,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-95 /mnt/projects/crispy_shifty/projects/crispy_crispies/06_mpnn_again/run.sh


### Collect scorefiles of the MPNN binders and concatenate

In [7]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "06_mpnn_again"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile

In [8]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

  0%|          | 0/18276 [00:00<?, ?it/s]

### Dump scores_df as a CSV and then reload, for performance reasons

In [9]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Data exploration

In [10]:
print(len(scores_df))
print(list(scores_df.columns))

18276
['bb_clash', 'bb_clash_delta', 'best_average_DAN_plddts', 'best_average_plddts', 'best_model', 'best_ptm', 'best_rmsd_to_input', 'buns_parent', 'chA_seq', 'cms', 'cms_AcB', 'cms_AnAc', 'cms_AnAcB', 'cms_AnB', 'count_apolar', 'ddg', 'designed_by', 'docked_helix', 'dslf_fa13', 'dssp', 'elapsed_time', 'exposed_hydrophobics_parent', 'fa_atr', 'fa_dun', 'fa_elec', 'fa_intra_rep', 'fa_intra_sol_xover4', 'fa_rep', 'fa_sol', 'geometry_parent', 'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb', 'holes_all_parent', 'holes_core_parent', 'insertion_dG', 'kept_end', 'kept_start', 'lk_ball_wtd', 'loop_dist', 'loop_sc', 'looped_length', 'mean_pae', 'mean_pae_interaction', 'mean_pae_interaction_AB', 'mean_pae_interaction_BA', 'mean_pae_intra_chain', 'mean_pae_intra_chain_A', 'mean_pae_intra_chain_B', 'mean_plddt', 'mismatch_probability_parent', 'model', 'mpnn_design_area', 'mpnn_seq_0000', 'mpnn_seq_0001', 'mpnn_seq_0002', 'mpnn_seq_0003', 'mpnn_seq_0004', 'mpnn_seq_0005', 'mpnn_seq_0006',

### Save individual fastas

In [11]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq")

  0%|          | 0/18276 [00:00<?, ?it/s]

### Save a list of outputs
Sort by length

In [12]:
simulation_name = "06_mpnn_again"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

scores_df = scores_df.sort_values("total_length")

with open(os.path.join(output_path, "mpnn_binders.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

  0%|          | 0/18276 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [13]:
simulation_name = "06_mpnn_again"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

with open(os.path.join(output_path, "mpnn_binders.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/18276 [00:00<?, ?it/s]